In [2]:
import random
import uuid
from pathlib import Path

import pandas as pd

import numpy as np

from librosa.core import get_duration, resample, load
from librosa.output import write_wav

from pydub import AudioSegment

from tqdm.autonotebook import tqdm, trange

In [3]:
import IPython.display as ipd

In [4]:
data_dir = Path('Y:\CSC\Audio-denoiser\data') / 'nr'

In [5]:
base_output_dir = Path('Y:\CSC\dual-path-rnn\data')

In [6]:
list(data_dir.iterdir())

[WindowsPath('Y:/CSC/Audio-denoiser/data/nr/test'),
 WindowsPath('Y:/CSC/Audio-denoiser/data/nr/train')]

## Почему так?
Из-за непонятной мне причины, `spleeter` работает только с стерое-аудио, причем с `sample_rate` только 44100.

In [7]:
from_sr = 8000
to_sr = 8000
to_channels = "1"

In [8]:
def preprocess_part(part, num_audios):
    df = pd.DataFrame(columns=["mix_path", "first_speaker_path", "second_speaker_path", "duration"])
    
    source_audio_dir = data_dir / part / "source"
    source_audios = list(source_audio_dir.iterdir())
    
    for i in trange(num_audios):
        # select 2 random audios to mix
        first_speaker_filename, second_speaker_filename = random.sample(source_audios, k=2)
        # read audios
        first_speaker = AudioSegment.from_wav(first_speaker_filename / "1.wav")
        second_speaker = AudioSegment.from_wav(second_speaker_filename / "1.wav")
        
        # random offset from start for mix
        offset_duration = random.randint(0, 5) * 1000 # seconds to milliseconds
        first_speaker += AudioSegment.silent(duration=offset_duration, frame_rate=to_sr)
        second_speaker = AudioSegment.silent(duration=offset_duration, frame_rate=to_sr) + second_speaker
        
        # overlay 2 audios
        mix_audio = first_speaker.overlay(second_speaker)
        
        # save to random output folder
        output_folder = base_output_dir / part / str(uuid.uuid4().hex)
        output_folder.mkdir(parents=True, exist_ok=True)
        first_speaker.export(output_folder / "first_speaker.wav", parameters=["-ac", to_channels], format="wav")
        second_speaker.export(output_folder / "second_speaker.wav", parameters=["-ac", to_channels], format="wav")
        mix_audio.export(output_folder / "mix_audio.wav", parameters=["-ac", to_channels], format="wav")
        
        # append to dataframe
        relative_path = output_folder.relative_to(base_output_dir)
        df.loc[len(df)] = [str(relative_path / "mix_audio.wav"), str(relative_path / "first_speaker.wav"), str(relative_path / "second_speaker.wav"), mix_audio.duration_seconds]
        
    df.to_csv(base_output_dir / f"{part}.csv", index=False)
    return df.head()

In [9]:
preprocess_part("train", num_audios=5000)

,mix_path,first_speaker_path,second_speaker_path,duration
0,train\07364d009bc647cb803d23dbce2c7168\mix_aud...,train\07364d009bc647cb803d23dbce2c7168\first_s...,train\07364d009bc647cb803d23dbce2c7168\second_...,16.0
1,train\3ae96017161a4695b169da102281970a\mix_aud...,train\3ae96017161a4695b169da102281970a\first_s...,train\3ae96017161a4695b169da102281970a\second_...,17.0
2,train\0a19908f247f46e9862de3730a9870f3\mix_aud...,train\0a19908f247f46e9862de3730a9870f3\first_s...,train\0a19908f247f46e9862de3730a9870f3\second_...,16.0
3,train\77b77c73022b4abc84ee704a95cb25c7\mix_aud...,train\77b77c73022b4abc84ee704a95cb25c7\first_s...,train\77b77c73022b4abc84ee704a95cb25c7\second_...,19.0
4,train\a677e6ea7a9d43029d109164f96f3b56\mix_aud...,train\a677e6ea7a9d43029d109164f96f3b56\first_s...,train\a677e6ea7a9d43029d109164f96f3b56\second_...,19.0


In [11]:
preprocess_part("test", num_audios=800)

,mix_path,first_speaker_path,second_speaker_path,duration
0,test\d88e8dad4b1c48e6b6d4753bc5c0ed4f\mix_audi...,test\d88e8dad4b1c48e6b6d4753bc5c0ed4f\first_sp...,test\d88e8dad4b1c48e6b6d4753bc5c0ed4f\second_s...,18.0
1,test\1a3018d9f46f49a480de5af431dd2f77\mix_audi...,test\1a3018d9f46f49a480de5af431dd2f77\first_sp...,test\1a3018d9f46f49a480de5af431dd2f77\second_s...,15.0
2,test\0dc27dfdb13c4e138e5bffab7ab7da66\mix_audi...,test\0dc27dfdb13c4e138e5bffab7ab7da66\first_sp...,test\0dc27dfdb13c4e138e5bffab7ab7da66\second_s...,19.0
3,test\afaf75938aa04dd38f98438234cdbefb\mix_audi...,test\afaf75938aa04dd38f98438234cdbefb\first_sp...,test\afaf75938aa04dd38f98438234cdbefb\second_s...,19.0
4,test\00e211cee7c94b588b812f442c4171c0\mix_audi...,test\00e211cee7c94b588b812f442c4171c0\first_sp...,test\00e211cee7c94b588b812f442c4171c0\second_s...,15.0
